In [18]:
# !pip install -q langchain==0.1.0 langchain-community==0.0.12
# !pip install -q langchainhub==0.1.14
# !pip install -q faiss-cpu faiss-gpu
# !pip install -U langsmith
# !pip install -q pandas
# !pip install -q python-dotenv
# !pip install --upgrade --quiet  langchain-google-genai
# !pip install faiss-cpu

In [1]:
import pandas as pd
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.document_loaders.dataframe import DataFrameLoader
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI

try:
    from google.colab.userdata import get as getenv
    print("Running in colab")
except ImportError:
    from os import getenv
    import dotenv
    dotenv.load_dotenv()

c:\Users\user\Desktop\RestoRecommender\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
GOOGLE_API_KEY = getenv('GOOGLE_API_KEY')

## Set up LangSmith

In [3]:
LANGCHAIN_API_KEY = getenv('LANGCHAIN_API_KEY')
LANGCHAIN_ENDPOINT = getenv('LANGCHAIN_ENDPOINT')
assert LANGCHAIN_API_KEY, "An API key for LangChainSmith is required to be set as <LANGCHAIN_API_KEY>."

## Constants

In [4]:
# Dataset files
PLACES_PATH = "data/places.csv"
REVIEWS_PATH = "data/reviews.csv"


LLM_MODEL_NAME = "gemini-1.5-flash" #"gemini-pro"
EMBEDDING_MODEL_NAME = "models/text-embedding-004"

# Embeddings
EMBEDDINGS_CACHE_STORE="./cache/"

# Faiss
FAISS_REVIEWS_PATH_EUCLIDEAN = "faiss_index_euclidean"
FAISS_REVIEWS_PATH_COSINE = "faiss_index_cosine"
FAISS_INDEX_NAME = "index"
FAISS_DISTANCE_STRATEGY_EUCLIDEAN ='EUCLIDEAN_DISTANCE'
FAISS_DISTANCE_STRATEGY_COSINE = "COSINE_DISTANCE"

## Load Dataset

Here we are using 2 csv files containing places (restuarants, bars, ...) info and reviews for each of them.

In [5]:
def get_documents(content_func=lambda row:row['review'],
                  source_func=lambda row:row['place_id'],
                  metadata_fields=[]):

  # Load both data files
  places_df = pd.read_csv(PLACES_PATH)
  reviews_df = pd.read_csv(REVIEWS_PATH)
  reviews_df.drop_duplicates(inplace=True)
  places_df.drop_duplicates(inplace=True)

  # merge them on 'place_id'
  merged_df = pd.merge(places_df, reviews_df, on='place_id', how='inner')

  # add page_content and source columns using their corresponing functions
  merged_df['page_content'] = merged_df.apply(content_func, axis=1)
  merged_df['source'] = merged_df.apply(source_func, axis=1)

  # update metadata_fields with 'page_content', 'source'
  metadata_fields = list(set(metadata_fields + ['page_content', 'source']))

  loader = DataFrameLoader(merged_df[metadata_fields],page_content_column='page_content')
  return loader.load()

In [6]:
# def content_func(row) -> str:
#   content_fields = ['place_name', 'place_types', 'place_address', 'place_average_ratings', 'review']
#   return '\n'.join(f"{key}={row[key]}" for key in content_fields)

def content_func(row) -> str:
  content_fields = ["place_name",
                    "review",
                    "place_address",
                    "place_average_ratings",
                    "place_price_level",
                    "place_primary_type",
                    "has_delivery",
                    "is_reservable",
                    ]
  return '\n'.join(f"{key}={row[key]}" for key in content_fields)

metadata_fields = ["places_opening_hours", "place_website", "place_phone_number",]

documents = get_documents(content_func, metadata_fields=metadata_fields)

In [28]:
## Take a look at a sample document
#print(documents[0].page_content)
print(documents[0].metadata)

{'source': 'ChIJORN_mdnbfkcRq7g9fQtODbE', 'places_opening_hours': "['Monday: Closed', 'Tuesday: 7:00\\u2009–\\u200911:00\\u202fPM', 'Wednesday: 7:00\\u2009–\\u200911:00\\u202fPM', 'Thursday: 12:00\\u2009–\\u20093:00\\u202fPM, 7:00\\u2009–\\u200911:00\\u202fPM', 'Friday: 12:00\\u2009–\\u20093:00\\u202fPM, 7:00\\u2009–\\u200911:00\\u202fPM', 'Saturday: 12:00\\u2009–\\u20093:00\\u202fPM, 7:00\\u2009–\\u200911:00\\u202fPM', 'Sunday: 12:00\\u2009–\\u20093:00\\u202fPM, 7:00\\u2009–\\u200911:00\\u202fPM']", 'place_website': nan, 'place_phone_number': '+39 347 360 4372'}


## Load Embeddings model

In [7]:
embedding_model = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL_NAME)

In [21]:
result = embedding_model.embed_query("One sample query!")

In [22]:
result

[0.013445205986499786, 0.0329967699944973, -0.05730990320444107, -0.0063919429667294025, 0.050566721707582474, 0.05204957723617554, 0.055957403033971786, 0.0265039149671793, -0.024919943884015083, -0.027119487524032593, -0.03403162956237793, -0.013598410412669182, 0.051470912992954254, -0.004248139914125204, 0.05007483437657356, -0.0025721651036292315, 0.027495834976434708, 0.06033796817064285, -0.10426695644855499, -0.03967343643307686, 0.04659663513302803, -0.022009238600730896, -0.030126888304948807, -0.03678601607680321, 0.03226298838853836, -0.019800592213869095, 0.012412684969604015, 0.0036472331266850233, 0.028147641569375992, -0.0005324278026819229, 0.0010376356076449156, 0.028569789603352547, -0.00806627981364727, -0.03816157579421997, -0.007220468949526548, 0.02569526433944702, 0.003935983404517174, 0.00042540504364296794, -0.005441155284643173, -0.0658370703458786, -0.004332046955823898, 0.05903252214193344, -0.05452469736337662, 0.03064827062189579, -0.020227117463946342, -

In [9]:
import numpy as np
array = np.array(result)
print(f"embedding shape: {array.shape}\nembedding norm: {np.linalg.norm(array, ord=2)}")

embedding shape: (768,)
embedding norm: 0.9999993842289441


## Create FAISS (Vector Database)

In [14]:
def get_vector_database(documents, embedding_model, distance_strategy):

  vector_database = FAISS.from_documents(
      documents, embedding_model,
      distance_strategy= distance_strategy
      )
  return vector_database

In [18]:
import time
doclen = len(documents)
for batch in range(doclen//100 + 1):
    docs = documents[batch*100:(batch+1)*100]
    if batch ==0:
        vector_db = get_vector_database(docs, embedding_model, FAISS_DISTANCE_STRATEGY_COSINE)
    else:

        vector_db.merge_from(get_vector_database(docs, embedding_model, FAISS_DISTANCE_STRATEGY_COSINE))
    time.sleep(10) # Sleep for 10 seconds to avoid hitting rate limits

In [20]:
vector_db.save_local(folder_path=FAISS_REVIEWS_PATH_COSINE, index_name=FAISS_INDEX_NAME)

In [10]:
vector_db = FAISS.load_local(folder_path=FAISS_REVIEWS_PATH_EUCLIDEAN,
                             embeddings=embedding_model,
                             index_name=FAISS_INDEX_NAME
                             )

In [22]:
docs = vector_db.similarity_search("Give me information about some of the best pizza restaurant in the city?", k = 5)
for doc in docs:
    print(doc, end="\n\n")

page_content='place_name=Pizzeria Al Quadrifoglio\nreview=Excellent pizza, now I consider it as the comparison for all the other pizzas I order in other places, unfortunately it is always unattainable. Congratulations to the head pizza chef even if he has few words.\nplace_address=Via Dante Alighieri, 4/a, 35020 Ponte San Nicolò PD, Italy\nplace_average_ratings=4.2\nplace_price_level=PRICE_LEVEL_INEXPENSIVE\nplace_primary_type=pizza_restaurant\nhas_delivery=True\nis_reservable=True' metadata={'places_opening_hours': "['Monday: Closed', 'Tuesday: 6:00\\u2009–\\u200910:00\\u202fPM', 'Wednesday: 6:00\\u2009–\\u200910:00\\u202fPM', 'Thursday: 6:00\\u2009–\\u200910:00\\u202fPM', 'Friday: 6:00\\u2009–\\u200910:00\\u202fPM', 'Saturday: 6:00\\u2009–\\u200910:00\\u202fPM', 'Sunday: 6:00\\u2009–\\u200910:00\\u202fPM']", 'place_website': 'https://m.facebook.com/PizzeriaAlQuadrifoglio/', 'place_phone_number': '+39 049 717742', 'source': 'ChIJnS7Ewr7EfkcRqlfUIy8hk6c'}

page_content='place_name=Rist

In [23]:
question = "where is the Enoteca Barcollo located? and what is its phone number?"

docs = vector_db.similarity_search(question, k = 5)

for i in range(5):
  print(docs[i], end="\n\n")

page_content='place_name=Enoteca Barcollo\nreview=The only positive thing is the location. For the rest, exorbitant prices, very small doses in microscopic wine glasses, 20cc Coca-Cola, supermarket Ichnusa beers, bread to be paid for separately and no appetizers. The two owners brought their rudeness with them from the winter headquarters. Over the years we have sometimes decided to go to the place, even knowing that there was always the risk of being mistreated, because they had good wine. Now the wine is no longer good and they are so arrogant with customers that it is offensive. Anyone who pays hard cash has the right to at least respect; wanting to be subtle, he would also be entitled to suitable portions, pleasant flavors and proportionate prices.\nplace_address=Via Alessandro Guidi, 23, 35142 Padova PD, Italy\nplace_average_ratings=4.3\nplace_price_level=PRICE_LEVEL_MODERATE\nplace_primary_type=bar\nhas_delivery=False\nis_reservable=True' metadata={'places_opening_hours': "['Mond

## Load Vector Database

In [20]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from os import getenv
import dotenv
dotenv.load_dotenv()


# Faiss
FAISS_REVIEWS_PATH_EUCLIDEAN = "faiss_index_euclidean"
FAISS_INDEX_NAME = "index"
FAISS_DISTANCE_STRATEGY='EUCLIDEAN_DISTANCE'
EMBEDDING_MODEL_NAME = "models/text-embedding-004"
EMBEDDINGS_CACHE_STORE="./cache/"

GOOGLE_API_KEY = getenv('GOOGLE_API_KEY')

embedding_model = GoogleGenerativeAIEmbeddings(model=EMBEDDING_MODEL_NAME)
store = LocalFileStore(EMBEDDINGS_CACHE_STORE)
embedding_model = CacheBackedEmbeddings.from_bytes_store(embedding_model, store)

vector_db = FAISS.load_local(folder_path=FAISS_REVIEWS_PATH_EUCLIDEAN,
                             embeddings=embedding_model,
                             index_name=FAISS_INDEX_NAME,
                             #allow_dangerous_deserialization=True
                             )

## Load LLM

In [12]:
llm = ChatGoogleGenerativeAI(model=LLM_MODEL_NAME)

In [27]:
llm.invoke("Hi")

AIMessage(content='Hi! How can I help you today? \n', response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HATE_SPEECH', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_HARASSMENT', 'probability': 'NEGLIGIBLE', 'blocked': False}, {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT', 'probability': 'NEGLIGIBLE', 'blocked': False}]}, id='run-dde23adf-e6d9-4920-8bec-de7d4da50483-0')

## Create LangChain pipeline

In [13]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough

In [25]:
review_template_str = """
Your job is to use Google Map restaurants and bars reviews to help people find best places to go for a meal or a drink.
Use the following information and reviews to answer the questions. if the context is not about restaurants,
then kindly tell the user that you can only provide assistance and answer questions related to restaurants.
If you don't know an answer based on the context, say you don't know. Answer context:
{context}
"""

system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_template_str
    )
)

human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [system_prompt, human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

reviews_retriever = vector_db.as_retriever(search_kwargs={'k': 20,})

review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | llm
    | StrOutputParser()
)

## Sample usage

In [26]:
question = """Where can I find delicious pizzas?"""
print(review_chain.invoke(question))

Here are some highly-rated pizzerias based on Google Maps reviews:

* **Officina Della Pizza:** This pizzeria is known for its delicious pizzas with high-quality ingredients and friendly service. They are located at Via Vittorio Bachelet, 35, 35010 Vigonza PD, Italy. 
* **Pizzeria Al Quadrifoglio:**  This pizzeria is known for its excellent pizza, quick service, and friendly staff. They are located at Via Dante Alighieri, 4/a, 35020 Ponte San Nicolò PD, Italy. 
* **da Pino Padova:** This pizza restaurant offers a delightful dining experience with high-quality pizzas and a warm atmosphere. They are located at Piazza Camillo Benso Conte di Cavour, 20, 35122 Padova PD, Italy.
* **pizzeria la poesia:** This pizzeria is known for its incredibly delicious pizzas and friendly service, with the owner speaking English. They are located at Via Piovese, 13, 35127 Padova PD, Italy. 
* **Crazy Pizza:** This pizzeria has a wood-burning oven, uses fresh products, and offers various pizza types and do

In [31]:
question = """Where can I find delicious pizzas?"""
print(review_chain.invoke(question))

Here are some highly-rated pizzerias according to Google Maps reviews:

* **Officina Della Pizza:**  Reviewers rave about the quality ingredients and the friendly service. It's located at Via Vittorio Bachelet, 35, 35010 Vigonza PD, Italy.
* **da Pino Padova:** This pizzeria is known for its perfectly cooked crust, flavorful sauce, and fresh toppings. It's located at Piazza Camillo Benso Conte di Cavour, 20, 35122 Padova PD, Italy.
* **Pizzeria Al Quadrifoglio:** This pizzeria has excellent pizza, quick service, and very friendly staff. It's located at Via Dante Alighieri, 4/a, 35020 Ponte San Nicolò PD, Italy.
* **Crazy Pizza:** This pizzeria offers a wood-burning oven, fresh products, and a wide variety of pizzas. They also cater to customers with allergies. It's located at Via Jacopo Filiasi, 2F, 35128 Padova PD, Italy.
* **pizzeria la poesia:** Reviewers say the pizza here is "the most delicious" they've ever had. The owner is known for his humor and English speaking skills. It's l

In [32]:
question = """What are the pros and cons of Napoli Centrale?"""
print(review_chain.invoke(question))

Here are some pros and cons of Ristorante Napoli Centrale based on the reviews:

**Pros:**

* **Delicious food:**  Reviewers consistently praise the pizza, with mentions of tasty dishes like grilled branzino, seafood pasta, and desserts. 
* **Excellent service:**  The staff is described as friendly, attentive, and unobtrusive.
* **Good value for money:** The price level is moderate, making it a good option for a satisfying meal without breaking the bank.

**Cons:**

* **No cocktail menu:** While they offer Aperol and Campari Spritz, there's no dedicated cocktail list.
* **Pizza fritti can be too much food:**  One reviewer found the pizza fritti to be a bit overwhelming in size, although delicious.

Overall, Ristorante Napoli Centrale seems like a good choice for a pleasant dining experience with delicious food and great service. 



In [16]:
question = """Give the name, address and phone number of some good steak houses for a romantic dinner."""
print(review_chain.invoke(question))

Here are some good steak houses in Padova that would be great for a romantic dinner:

* **Ristorante Vecchio Falconiere:**  Via Umberto I, 31, 35122 Padova PD, Italy. Phone: +39 049 656544.  Reviewers rave about the passion for food, the friendly owner, and the table-side steak cooking. They have a large selection of steaks and wines.
* **Osteria BocaBona:** Str. Pelosa, 4, 35136 Padova PD, Italy. Phone: +39 049 871 3898.  This place is known for its welcoming atmosphere and excellent selection of well-matured and cooked meats. The beef steak is a standout.
* **La Fiorentina:** Via G. Marconi, 123, 35020 Ponte San Nicolò PD, Italy. Phone: +39 049 816 4672.  Reviewers praise the quality of the meat, the friendly owners, and the cozy ambiance. The steak is cooked to perfection and very tender. 
* **Osteria Nonna Pina:** Via Roma, 65, 35027 Noventa Padovana PD, Italy. Phone: +39 049 830 6301.  This restaurant has a stylish interior, outdoor grill, and serves some of the most delicious mea

In [67]:
question = """Give the name, address and phone number of the best steak houses with a 50 euro budget?"""

print(review_chain.invoke(question))

Based on the reviews provided, the best steak house option with a 50 euro budget seems to be Ristorante Vecchio Falconiere. Here are the details:

Ristorante Vecchio Falconiere
Address: Via Umberto I, 31, 35122 Padova PD, Italy 
Phone: +39 049 656544

The review mentions they have a large variety of steaks in different sizes, from local to Wagyu beef. It's described as a "gem for meat and wine lovers". However, the review doesn't specify prices, so I can't say for certain if it would fit a 50 euro per person budget. The other restaurants reviewed don't appear to specialize in steak.


In [26]:
question = """Give the name, address and phone number of the some good sandwich places?"""
print(review_chain.invoke(question))

Based on the reviews, a couple good sandwich places are:

Soul Kitchen
Via del Santo, 23, 35123 Padova PD, Italy
+39 049 664818

The reviews mention they have great burgers, including an Angus beef burger with excellent onion rings. One review wasn't as impressed with their carbonara burger though.

Spiller | Padova 
Via Fornace Morandi, 24/B, 35133 Padova PD, Italy
+39 049 864 4822

A review recommends their club sandwich and says the location is nice with outdoor seating available. They suggest booking in advance as it can get busy.

The other places focus more on burgers rather than a variety of sandwiches. I don't have enough information to confidently recommend additional sandwich spots beyond those two based on the reviews provided.


In [37]:
question = """What are the most affordable but high-quality restaurants in City?"""
result = review_chain.invoke(question)
print(result)

Based on the reviews provided, the most affordable yet high-quality restaurant in Padova seems to be Peace 'n' Spice. A few key points from the reviews:

- It offers a variety of delicious Middle Eastern dishes at an inexpensive price level. One reviewer said "Food was amazing. Very delicious, every dish we took was very tasty."

- The ambiance and decor are described as fantastic. One review mentioned "The ambiance is fantastic and even music in background brings nostalgia."

- Service is friendly and accommodating. A reviewer said "Staffs are very friendly and polite."

- It has an average rating of 4.5 stars, higher than the McDonald's and KFC locations mentioned which are in the 3-4 star range.

The McDonald's and KFC have mixed reviews, with some praising the taste/quality but others feeling it has declined. They are also inexpensive options.

So in summary, for an affordable restaurant in Padova with high quality food, friendly service and nice atmosphere, Peace 'n' Spice stands 

In [72]:
question = """How can I make a roast beef sandwich at home?"""
print(review_chain.invoke(question))

I apologize, but I did not find information about how to make a roast beef sandwich at home in the provided reviews. The reviews are about various restaurants in Padova, Italy, including Hamerica's, Autogrill Limenella, Old England Pub, Ristorante Vecchio Falconiere, Brunch Republic Padova, Bar Nazionale, and McDonald's Padova Ovest. They mention dishes like burgers, chicken burrito, vegan pulled pork sandwich, steak, and tramezzino sandwiches, but do not provide instructions for making a roast beef sandwich at home.


In [ ]:
question = """What is RAG?"""
print(review_chain.invoke(question))

I'm sorry, I can only answer questions related to restaurants and bars based on the provided reviews. The context doesn't contain information about RAG. 



In [ ]:
question = """What is Natural Language Processing?"""
print(review_chain.invoke(question))

I can only provide assistance and answer questions related to restaurants. 



In [ ]:
question = """Explain Natural Language Processing."""
print(review_chain.invoke(question))

I can only provide assistance and answer questions related to restaurants. I don't know anything about Natural Language Processing. 



## Evaluation on Synthetic Questions

In [34]:
from pprint import pprint
import random

In [76]:
def get_question_answer_pairs(documents, generator_llm, num_pairs=30):
  question_answer_pairs = []
  for _ in range(num_pairs):
    document = random.choice(documents)
    page_content = document.page_content
    prompt = f"This is a factual text passage: {page_content}. Write only one question about the restaurant based on the provided text passage. only write the quesion and noting else."

    question = generator_llm.invoke(prompt).content
    answer = generator_llm.invoke(f"From the following passage, answer the question: {question}\n{page_content}").content
    question_answer_pairs.append({"question": question, "answer": answer, "document": document})

  return question_answer_pairs

In [77]:
generator_llm = ChatGoogleGenerativeAI(model=LLM_MODEL_NAME)
question_answer_pairs = get_question_answer_pairs(documents, generator_llm, num_pairs=30)
df = pd.DataFrame(question_answer_pairs)

In [16]:
rag_answers = []
for question in df["question"]:
    question = question.split("\n")[0]
    answer = review_chain.invoke(question)
    rag_answers.append(answer)

df["rag_answer"] = rag_answers
df.to_csv('question_answer_pairs.csv', index=False)

In [21]:
df.to_csv('question_answer_pairs.csv', index=False)
df.head()

,question,answer,document,rag_answer
0,Does Bar Fortuna Sas offer delivery services? \n,The passage states that `has_delivery=False`. ...,page_content='place_name=Bar Fortuna Sas\nrevi...,"No, Bar Fortuna Sas does not offer delivery se..."
1,Is Crazy Pizza a good option for a budget-frie...,"Yes, based on the information provided, Crazy ...",page_content='place_name=Crazy Pizza\nreview=B...,"Yes, Crazy Pizza is a good option for a budget..."
2,What are the standout dishes at Ristorante da ...,The standout dishes at Ristorante da Giovanni ...,"page_content=""place_name=Ristorante da Giovann...",Ristorante da Giovanni is known for its authen...
3,Is Anima Underground a good place to find drag...,"Based on the provided information, **yes**, An...",page_content='place_name=Anima Underground\nre...,"Yes, Anima Underground is a good place to find..."
4,What kind of food is served at Veni Vidi Vino ...,"The passage mentions ""substantial and spectacu...","page_content=""place_name=Veni Vidi Vino Enotec...","Veni Vidi Vino Enoteca serves tasty food, incl..."


In [20]:
df = pd.read_csv('question_answer_pairs.csv')
df.head()

,question,answer,document,rag_answer
0,Does Bar Fortuna Sas offer delivery services? \n,The passage states that `has_delivery=False`. ...,page_content='place_name=Bar Fortuna Sas\nrevi...,"No, Bar Fortuna Sas does not offer delivery se..."
1,Is Crazy Pizza a good option for a budget-frie...,"Yes, based on the information provided, Crazy ...",page_content='place_name=Crazy Pizza\nreview=B...,"Yes, Crazy Pizza is a good option for a budget..."
2,What are the standout dishes at Ristorante da ...,The standout dishes at Ristorante da Giovanni ...,"page_content=""place_name=Ristorante da Giovann...",Ristorante da Giovanni is known for its authen...
3,Is Anima Underground a good place to find drag...,"Based on the provided information, **yes**, An...",page_content='place_name=Anima Underground\nre...,"Yes, Anima Underground is a good place to find..."
4,What kind of food is served at Veni Vidi Vino ...,"The passage mentions ""substantial and spectacu...","page_content=""place_name=Veni Vidi Vino Enotec...","Veni Vidi Vino Enoteca serves tasty food, incl..."


In [40]:
for i in range(10): 
    print(df["question"].loc[i])

Does Bar Fortuna Sas offer delivery services? 

Is Crazy Pizza a good option for a budget-friendly meal? 

What are the standout dishes at Ristorante da Giovanni? 

Is Anima Underground a good place to find drag performances? 

What kind of food is served at Veni Vidi Vino Enoteca? 

Does Crazy Pizza offer dine-in options? 

Does Osteria Di Fuori Porta offer delivery? 

Is Bar La Mandria in Padova, Italy, reservable? 

Is Osteria Enoteca L'Indeciso a moderately priced restaurant in Padua, Italy? 

Is San Sushi a popular choice for sushi in Padova? 



## Limitatoins of classic LLMs

In [ ]:
question = "Does Bar Fortuna Sas in Padova city offer delivery services?"
answer = llm.invoke(question).content
pprint(question)
pprint(answer)

'Does Bar Fortuna Sas in Padova city offer delivery services?'
('I do not have access to real-time information, including business details '
 'like delivery services. To find out if Bar Fortuna Sas in Padova city offers '
 'delivery, I recommend checking their:\n'
 '\n'
 '* **Website:** Many businesses have websites that list their services.\n'
 '* **Social Media:** Look for their Facebook or Instagram page, as they may '
 'advertise delivery options there.\n'
 '* **Online Food Delivery Apps:** Check apps like Deliveroo, Uber Eats, or '
 'Glovo to see if Bar Fortuna Sas is listed.\n'
 '* **Phone Call:** Call the restaurant directly to inquire about their '
 'delivery options. \n'
 '\n'
 'I hope this helps! \n')


In [24]:
question = "Does Bar Fortuna Sas in Padova city offer delivery services?"
answer = review_chain.invoke(question)
print(question)
print(answer)

Does Bar Fortuna Sas offer delivery services? 

The passage states that `has_delivery=False`. Therefore, Bar Fortuna Sas **does not** offer delivery services. 

page_content='place_name=Bar Fortuna Sas\nreview=Excellent breakfast.....\nplace_address=Via Adriatica, 21, 35125 Padova PD, Italy\nplace_average_ratings=3.9\nplace_price_level=PRICE_LEVEL_INEXPENSIVE\nplace_primary_type=bar\nhas_delivery=False\nis_reservable=False' metadata={'place_phone_number': '+39 049 673 2077', 'place_website': nan, 'source': 'ChIJscwimbzbfkcRAcJSXrAFVns', 'places_opening_hours': "['Monday: 6:30\\u202fAM\\u2009–\\u20092:00\\u202fAM', 'Tuesday: 3:00\\u202fPM\\u2009–\\u20092:00\\u202fAM', 'Wednesday: 6:30\\u202fAM\\u2009–\\u20092:00\\u202fAM', 'Thursday: 6:30\\u202fAM\\u2009–\\u20092:00\\u202fAM', 'Friday: 6:30\\u202fAM\\u2009–\\u20092:00\\u202fAM', 'Saturday: 6:30\\u202fAM\\u2009–\\u20092:00\\u202fAM', 'Sunday: 6:30\\u202fAM\\u2009–\\u20092:00\\u202fAM']", 'filename': 'ChIJscwimbzbfkcRAcJSXrAFVns'}
No, Bar

In [41]:
question = "What kind of food is served at Veni Vidi Vino Enoteca?"
answer = llm.invoke(question).content
pprint(question)
pprint(answer)

'What kind of food is served at Veni Vidi Vino Enoteca?'
('I do not have access to real-time information, including menus for '
 'restaurants. To find out what kind of food is served at Veni Vidi Vino '
 'Enoteca, I recommend checking their website or calling the restaurant '
 'directly. \n')


In [43]:
question = "What kind of food is served at Veni Vidi Vino Enoteca?"
answer = review_chain.invoke(question)
pprint(question)
pprint(answer)

'What kind of food is served at Veni Vidi Vino Enoteca?'
('Veni Vidi Vino Enoteca serves very tasty food, according to one review.  '
 'It\'s described as "too much for us to eat."  The review also mentions "top '
 'level wine" and describes the owner as a "very polite person." \n')
